In [57]:
import pandas as pd
import numpy as np
import sklearn
import sklearn.ensemble
from sklearn.metrics import accuracy_score
from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

In [58]:
def preproceesing_data(df):
  #removing rows with loan_status as current
  df.drop(df.index[df['loan_status'] == 'Current'], inplace = True)

  #renaming labels and assigning values to target varibles
 # train_data.rename(columns = {'loan_status':'label'}, inplace = True)
  df['loan_status'].replace({"Charged Off": -1, "Fully Paid": +1}, inplace=True)

  #dropping columns with all null values
  df=df.dropna(axis='columns',how="all")

  #dropping columns with only single value
  for col in df.columns:
      if len(df[col].unique()) == 1:
          df.drop(col,inplace=True,axis=1)

  #droping columns wich have no useful information like id, adress, url and description
  df=df.drop(['url','desc','id','member_id','zip_code','title','emp_title'],axis=1)
  # df=df.drop(['funded_amnt','total_rec_prncp','recoveries'],axis=1)
  #removing columns which have null values more than 20% of the training data
  for col in df:
    if df[col].isnull().sum() > (df.shape[0])/5 :
      df.drop(col,axis=1,inplace=True)

  #removing rows which have null values
  df=df.dropna(axis='rows')

  #dropping columns with only single value
  for col in df.columns:
      if len(df[col].unique()) == 1:
          df.drop(col,inplace=True,axis=1)

  #corr = train_df.corr()['loan_status'].sort_values()
  #print('\nMost Negative Correlations:\n', corr.head(10))
  least_core=['recoveries','collection_recovery_fee','sub_grade','grade','term','total_rec_late_fee','revol_util','inq_last_6mths','loan_amnt','funded_amnt','purpose','emp_length','addr_state','home_ownership']
  df=df.drop(least_core,axis=1)

  print(df.shape)

  for col_name in df.columns:
    if(df[col_name].dtype == 'object'):
        df[col_name]= df[col_name].astype('category')
        df[col_name] = df[col_name].cat.codes

  return df

In [59]:
#getting data provide path
train_data = pd.read_csv('/content/drive/MyDrive/FOML/loan_train.csv', sep=",")   
test_data = pd.read_csv('/content/drive/MyDrive/FOML/loan_test.csv', sep=",")

In [60]:
#preprocessing_data

train_df=preproceesing_data(train_data)
test_df=preproceesing_data(test_data)

train_X_df=train_df.drop(['loan_status'],axis=1)
test_X_df=test_df.drop(['loan_status'],axis=1)

train_X=train_X_df.to_numpy()
train_Y=train_df['loan_status'].to_numpy()
test_X=test_X_df.to_numpy()
test_Y=test_df['loan_status'].to_numpy()

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


(23771, 22)
(12964, 22)


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [61]:
#feature selection 
#classifying 
'''
clf = GradientBoostingClassifier(n_estimators=300, learning_rate=0.1,criterion='mse',max_depth=4, random_state=10,max_features='auto').fit(train_X, train_Y)
clf.score(test_X, test_Y)
'''
#calculating importance of features and removing features with lower importance

'''
features=pd.DataFrame({"Variable_Name":train_X_df.columns,"Importance":clf.feature_importances_}).sort_values('Importance', ascending=False)
y=features.Importance.mean()
list=[]
for index,feature in features.iterrows():
    if feature["Importance"]<y:
      list.append(feature["Variable_Name"])

'''
#calculated the list and removing features
#list=['last_pymnt_amnt','last_credit_pull_d','total_rec_int','last_pymnt_d','loan_amnt','total_rec_late_fee','installment','total_pymnt','funded_amnt_inv','emp_title','revol_bal','annual_inc','open_acc','total_pymnt_inv','sub_grade','dti','purpose','earliest_cr_line','revol_util','emp_length','issue_d','delinq_2yrs','inq_last_6mths','addr_state','int_rate','total_acc','grade','collection_recovery_fee','home_ownership','pub_rec','verification_status','pub_rec_bankruptcies','term']
                
'''
modified_train_X_df=train_X_df.drop(list,axis=1)
modified_test_X_df=test_X_df.drop(list,axis=1)
modified_train_X=modified_train_X_df.to_numpy()
modified_test_X=modified_test_X_df.to_numpy()'''

'\nmodified_train_X_df=train_X_df.drop(list,axis=1)\nmodified_test_X_df=test_X_df.drop(list,axis=1)\nmodified_train_X=modified_train_X_df.to_numpy()\nmodified_test_X=modified_test_X_df.to_numpy()'

In [62]:
#classifying on important features

clf = GradientBoostingClassifier(n_estimators=400, learning_rate=0.2,criterion='friedman_mse',max_depth=4, random_state=10,max_features='auto').fit(train_X, train_Y)
print(clf.score(test_X, test_Y))
print(metrics.precision_score(test_Y, clf.predict(test_X)))
print(metrics.recall_score(test_Y, clf.predict(test_X)))

0.994214748534403
0.9933751119068934
0.9999098855546544


In [63]:
#inbuilt classifier

clf_dt = DecisionTreeClassifier(criterion = "entropy",random_state = 10,max_depth=3).fit(train_X, train_Y)
print(clf_dt.score(test_X, test_Y))
print(metrics.precision_score(test_Y, clf_dt.predict(test_X)))
print(metrics.recall_score(test_Y, clf_dt.predict(test_X)))

0.9288799753162604
0.9233585753515853
0.9999098855546544
